In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [32]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [33]:
driver.get("https://www.koreabaseball.com/Default.aspx?vote=true")

In [34]:
driver.find_element(By.CSS_SELECTOR,"#lnb > li:nth-child(3) > a").click()

In [ ]:
team_selected = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"

In [ ]:
driver.find_element(By.CSS_SELECTOR, team_selected.format(5)).click()

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
tmp = []
for x in BeautifulSoup(driver.page_source).find("div", class_='paging').find_all("a"):
    if x.text != '':
        tmp.append(int(x.text))

In [ ]:
import time 
team_selected = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"
page_selected = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"
for numb in range(2, 12):
    driver.find_element(By.CSS_SELECTOR, team_selected.format(numb)).click()
    time.sleep(5)
    last_number = max([int(x.text) for x in BeautifulSoup(driver.page_source).find("div", class_='paging').find_all("a") if x.text != ''])
    print(f"{numb} --> last number : {last_number}")
    for page in range(2,last_number+1):
        driver.find_element(By.CSS_SELECTOR, page_selected.format(page)).click()
        time.sleep(3)

2 --> last number : 5
3 --> last number : 5
4 --> last number : 5
5 --> last number : 4
6 --> last number : 5


KeyboardInterrupt: 

In [36]:
bs = BeautifulSoup(driver.page_source)

In [39]:
def get_page_info(bs):
    total = list()
    for data in bs.find('table', class_='tEx').find_all('tr')[1:]:
        play_list = []
        for idx, x in enumerate(data.find_all("td")):
            if idx == 1:
                play_list.extend([x.text,x.find('a')['href'].split("=")[-1]])
                # print(x.text, x.find('a')['href'].split("=")[-1])
            else:
                play_list.append(x.text)


        total.append(play_list)
    return total 

In [43]:
import time 
team_selected = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"
page_selected = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"
total_result = []
for numb in range(2, 12):
    driver.find_element(By.CSS_SELECTOR, team_selected.format(numb)).click()
    time.sleep(3)
    last_number = max([int(x.text) for x in BeautifulSoup(driver.page_source).find("div", class_='paging').find_all("a") if x.text != ''])
    # print(f"{numb} --> last number : {last_number}")
    total_result.extend(get_page_info(BeautifulSoup(driver.page_source)))
    for page in range(2,last_number+1):
        driver.find_element(By.CSS_SELECTOR, page_selected.format(page)).click()
        time.sleep(3)
        total_result.extend(get_page_info(BeautifulSoup(driver.page_source)))

In [52]:
sql = """CREATE TABLE kbo_player (
    player_number int default null, 
    name varchar(30) not null, 
    kbo_id varchar(10)  not null,
    team varchar(10)  not null,
    position varchar(20) default null, 
    birthday date default null, 
    height int default null, 
    weight int default null,
    career varchar(50) default null
)"""

In [49]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='', passwd='', database='', port=3306)
#127.0.0.1 -> lookback ip

In [50]:
cursor = conn.cursor()

In [53]:
cursor.execute(sql)

0

In [46]:
import pandas as pd 
pd.DataFrame(total_result)

,0,1,2,3,4,5,6,7
0,100,강민,50105,LG,투수,2001-04-20,"188cm, 88kg",수유초-홍은중-서울고
1,117,강민균,53103,LG,내야수,2000-07-17,"178cm, 81kg",강남초-서울이수중-덕수고-홍익대
2,116,고영웅,55116,LG,투수,2004-07-29,"180cm, 90kg",태전초(경기광주시리틀)-모가중-야로고BC-동원대
3,124,곽민호,53104,LG,내야수,2004-12-23,"184cm, 90kg",역삼초-자양중-배명고
4,6,구본혁,69100,LG,내야수,1997-01-11,"177cm, 75kg",중대초-잠신중-장충고-동국대-LG-상무
...,...,...,...,...,...,...,...,...
888,30,주승우,52330,키움,투수,2000-01-30,"180cm, 82kg",송추초(의정부리틀)-영동중-서울고-성균관대
889,53,최주환,76267,키움,내야수,1988-02-28,"177cm, 73kg",학강초-광주동성중-광주동성고-두산-상무-두산-SK-SSG
890,4,카디네스,54444,키움,외야수,1997-10-10,"185cm, 93kg",미국 California State(대)-삼성
891,,푸이그,52366,키움,외야수,1990-12-07,"188cm, 108kg",쿠바 EIEFD(대)-키움


In [86]:
import pickle 
with open("./kbo_raw.pkl", "wb") as f:
    pickle.dump(total_result, f)

In [93]:
for data in total_result:
    height, weight = data[6].split(',')
    height = height.strip().replace("cm", "")
    weight = weight.strip().replace("kg", "")
    del data[6]
    data.insert(6,height)
    data.insert(7,weight)

In [103]:
query = "insert into kbo_player values(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
cursor.execute("truncate table kbo_player")
for row in total_result:
    cursor.execute(query, [x if x != '' else None for x in row] )
        


In [104]:
conn.commit()

In [ ]:
sql = """SELECT kbo_id 
FROM kbo_player"""
cursor.execute(sql)
cursor.fetchall()

893

(('50105',),
 ('53103',),
 ('55116',),
 ('53104',),
 ('69100',),
 ('77263',),
 ('66145',),
 ('54107',),
 ('67504',),
 ('53144',),
 ('52105',),
 ('69105',),
 ('50115',),
 ('55167',),
 ('68130',),
 ('55129',),
 ('64596',),
 ('68109',),
 ('55122',),
 ('66162',),
 ('65411',),
 ('51104',),
 ('75867',),
 ('51154',),
 ('76290',),
 ('54166',),
 ('51103',),
 ('69102',),
 ('68119',),
 ('52121',),
 ('54123',),
 ('55164',),
 ('79365',),
 ('53139',),
 ('51112',),
 ('55121',),
 ('62415',),
 ('63145',),
 ('68096',),
 ('65109',),
 ('55160',),
 ('68159',),
 ('55118',),
 ('54156',),
 ('67143',),
 ('53105',),
 ('51111',),
 ('68110',),
 ('65207',),
 ('54108',),
 ('61411',),
 ('55109',),
 ('65115',),
 ('52124',),
 ('54119',),
 ('53123',),
 ('79109',),
 ('51526',),
 ('55111',),
 ('53101',),
 ('50106',),
 ('51110',),
 ('51100',),
 ('61145',),
 ('96153',),
 ('50904',),
 ('52154',),
 ('53160',),
 ('69108',),
 ('55165',),
 ('55100',),
 ('53100',),
 ('69113',),
 ('61101',),
 ('63950',),
 ('68869',),
 ('67123',),